In [64]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import style
style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [66]:
train = pd.read_csv('/content/Movie Review.csv')
train.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,0
1,A wonderful little production. <br /><br />The...,0
2,I thought this was a wonderful way to spend ti...,0
3,Basically there's a family where a little boy ...,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",0


In [67]:
def remove_pattern(text,pattern):

    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)

    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)

    return text

In [68]:
train['Tidy_Reviews'] = np.vectorize(remove_pattern)(train['review'], "@[\w]*")
train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")
train['Tidy_Reviews'] = train['Tidy_Reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train.head()

<ipython-input-68-be17ecef7fa3>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")


,review,sentiment,Tidy_Reviews
0,One of the other reviewers has mentioned that ...,0,other reviewers mentioned that after watching ...
1,A wonderful little production. <br /><br />The...,0,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,0,thought this wonderful spend time summer weeke...
3,Basically there's a family where a little boy ...,1,Basically there family where little Jake think...
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,Petter Mattei Love Time Money visually stunnin...


In [69]:
tokenized_review = train['Tidy_Reviews'].apply(lambda x: x.split())
tokenized_review.head()

0    [other, reviewers, mentioned, that, after, wat...
1    [wonderful, little, production, filming, techn...
2    [thought, this, wonderful, spend, time, summer...
3    [Basically, there, family, where, little, Jake...
4    [Petter, Mattei, Love, Time, Money, visually, ...
Name: Tidy_Reviews, dtype: object

In [70]:
from nltk import PorterStemmer
ps = PorterStemmer()
tokenized_review = tokenized_review.apply(lambda x: [ps.stem(i) for i in x])
tokenized_review.head()

0    [other, review, mention, that, after, watch, j...
1    [wonder, littl, product, film, techniqu, veri,...
2    [thought, thi, wonder, spend, time, summer, we...
3    [basic, there, famili, where, littl, jake, thi...
4    [petter, mattei, love, time, money, visual, st...
Name: Tidy_Reviews, dtype: object

In [42]:
for i in range(len(tokenized_review)):
    tokenized_review[i] = ' '.join(tokenized_review[i])

train['Tidy_Reviews'] = tokenized_review
train.head()

,Review,Label,Tidy_Reviews
0,good and interesting,0,good interest
1,"This class is very helpful to me. Currently, I...",0,thi class veri help current still learn thi cl...
2,like!Prof and TAs are helpful and the discussi...,0,like prof help discuss among student quit acti...
3,Easy to follow and includes a lot basic and im...,0,easi follow includ basic import techniqu sketchup
4,Really nice teacher!I could got the point eazl...,0,realli nice teacher could point eazliy


In [71]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['Tidy_Reviews'])

df_bow = pd.DataFrame(bow.todense())

df_bow

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
4,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33995,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
33997,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
33998,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')
tfidf=tfidf.fit_transform(train['Tidy_Reviews'])
df_tfidf = pd.DataFrame(tfidf.todense())
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.114374,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.120901,0.00000,0.132269,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.355243,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.079180,0.105515,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.14956,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33995,0.0,0.0,0.0,0.000000,0.0,0.080557,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0
33996,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.076821,0.0,0.000000,0.0
33997,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.228524,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0
33998,0.0,0.0,0.0,0.178965,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0


In [73]:
train_bow = bow[:34000]

train_bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [74]:
train_tfidf= tfidf[:34000]

train_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [75]:
from sklearn.model_selection import train_test_split

In [76]:
x_train_bow,x_valid_bow,y_train_bow,y_valid_bow = train_test_split(train_bow,train['sentiment'],test_size=0.3,random_state=2)
x_train_tfidf,x_valid_tfidf,y_train_tfidf,y_valid_tfidf = train_test_split(train_tfidf,train['sentiment'],test_size=0.3,random_state=17)

In [77]:
pip install keras.preprocessing

In [78]:
from keras.preprocessing.text import one_hot, Tokenizer
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from numpy import array
import tensorflow as tf

In [79]:
x_train_bow = x_train_bow.toarray()
x_valid_bow = x_valid_bow.toarray()

In [80]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train_bow.shape[1]))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_bow, y_train_bow, batch_size=5, epochs=15)

Epoch 1/15
4760/4760 [==============================] - 22s 4ms/step - loss: 0.3917 - accuracy: 0.8342
Epoch 2/15
4760/4760 [==============================] - 34s 7ms/step - loss: 0.3467 - accuracy: 0.8591
Epoch 3/15
4760/4760 [==============================] - 29s 6ms/step - loss: 0.3397 - accuracy: 0.8648
Epoch 4/15
4760/4760 [==============================] - 17s 4ms/step - loss: 0.3376 - accuracy: 0.8653
Epoch 5/15
4760/4760 [==============================] - 16s 3ms/step - loss: 0.3365 - accuracy: 0.8668
Epoch 6/15
4760/4760 [==============================] - 16s 3ms/step - loss: 0.3348 - accuracy: 0.8658
Epoch 7/15
4760/4760 [==============================] - 17s 4ms/step - loss: 0.3313 - accuracy: 0.8670
Epoch 8/15
4760/4760 [==============================] - 15s 3ms/step - loss: 0.3318 - accuracy: 0.8661
Epoch 9/15
4760/4760 [==============================] - 13s 3ms/step - loss: 0.3306 - accuracy: 0.8682
Epoch 10/15
4760/4760 [==============================] - 15s 3ms/step - l

In [81]:
score = model.evaluate(x_valid_bow, y_valid_bow, verbose=1)

319/319 [==============================] - 2s 4ms/step - loss: 0.3605 - accuracy: 0.8512


In [82]:
x_train_tfidf = x_train_tfidf.toarray()
x_valid_tfidf = x_valid_tfidf.toarray()

In [83]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train_bow.shape[1]))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_tfidf, y_train_tfidf, batch_size=5, epochs=15)

Epoch 1/15
4760/4760 [==============================] - 29s 6ms/step - loss: 0.3894 - accuracy: 0.8285
Epoch 2/15
4760/4760 [==============================] - 30s 6ms/step - loss: 0.3254 - accuracy: 0.8613
Epoch 3/15
4760/4760 [==============================] - 11s 2ms/step - loss: 0.3189 - accuracy: 0.8630
Epoch 4/15
4760/4760 [==============================] - 11s 2ms/step - loss: 0.3166 - accuracy: 0.8661
Epoch 5/15
4760/4760 [==============================] - 11s 2ms/step - loss: 0.3153 - accuracy: 0.8668
Epoch 6/15
4760/4760 [==============================] - 10s 2ms/step - loss: 0.3143 - accuracy: 0.8668
Epoch 7/15
4760/4760 [==============================] - 11s 2ms/step - loss: 0.3137 - accuracy: 0.8681
Epoch 8/15
4760/4760 [==============================] - 11s 2ms/step - loss: 0.3135 - accuracy: 0.8694
Epoch 9/15
4760/4760 [==============================] - 11s 2ms/step - loss: 0.3119 - accuracy: 0.8701
Epoch 10/15
4760/4760 [==============================] - 11s 2ms/step - l

In [84]:
score = model.evaluate(x_valid_tfidf, y_valid_tfidf, verbose=1)

319/319 [==============================] - 1s 3ms/step - loss: 0.3597 - accuracy: 0.8506
